In [20]:
import pandas as panda
import numpy as np
from collections import Counter

In [2]:
hist_with_merchants_data_path = 'C:/TV Series/real_world_use_cases_1/supervised/regression/kaggle/elo/all_new/hist_with_merchant.csv'


In [3]:
hist_transaction_data = panda.read_csv(hist_with_merchants_data_path)
hist_transaction_data_types = hist_transaction_data.dtypes
hist_transaction_column_types = {}
for index, column_name in enumerate(hist_transaction_data.columns.tolist()):
    hist_transaction_column_types[column_name] = hist_transaction_data_types[index]
del hist_transaction_data
hist_transaction_data = panda.read_csv(hist_with_merchants_data_path, dtype = hist_transaction_column_types)
hist_transaction_data.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [5]:
hist_transaction_data.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
authorized_flag,28973880,2,Y,26475396,NaN,NaN,NaN,NaN,NaN,NaN,NaN
card_id,28973880,325540,C_ID_3d3dfdc692,5582,NaN,NaN,NaN,NaN,NaN,NaN,NaN
city_id,2.89739e+07,NaN,NaN,NaN,129.306,104.272,-1,53,90,212,347
category_1,28973880,2,N,26893188,NaN,NaN,NaN,NaN,NaN,NaN,NaN
installments,2.89739e+07,NaN,NaN,NaN,0.64988,2.80186,-1,0,0,1,999
category_3,28802928,3,A,15336465,NaN,NaN,NaN,NaN,NaN,NaN,NaN
merchant_category_id,2.89739e+07,NaN,NaN,NaN,481.078,249.363,-1,307,454,705,891
merchant_id,28973880,326311,M_ID_00a6ca8a8a,1115097,NaN,NaN,NaN,NaN,NaN,NaN,NaN
month_lag,2.89739e+07,NaN,NaN,NaN,-4.50404,3.58772,-13,-7,-4,-2,0
purchase_amount,2.89739e+07,NaN,NaN,NaN,0.0342455,1126.08,-0.746908,-0.720311,-0.688304,-0.603029,6.0106e+06


In [6]:
hist_transaction_data.isnull().sum()

authorized_flag               0
card_id                       0
city_id                       0
category_1                    0
installments                  0
category_3               170952
merchant_category_id          0
merchant_id                   0
month_lag                     0
purchase_amount               0
purchase_date                 0
category_2              2642831
state_id                      0
subsector_id                  0
dtype: int64

Observations from above:

1. state_id/ subsector_id /merchant_category_id/city_id has missing data, since i can see it has values which are - 1. We are going to leave them as is

2. installments has missing data , shown in the formats of both -1 and 999. we will take two strategies: replace both wiht mean and most common installment value to create two separate columns

3. month lag has only negative values, we need to look at it closely

4. category_3 / category_2 have both missing data. replacing it most common appearing data


In [7]:
hist_transaction_data.month_lag.value_counts()

-2     3835910
-1     3589436
 0     3404841
-3     3087434
-4     2637822
-5     2364232
-6     2049613
-7     1769489
-8     1427532
-9     1234466
-10    1084782
-11    1007695
-12     791748
-13     688880
Name: month_lag, dtype: int64

In [8]:
def prep_for_aggregate(table):
    
    table['authorized_flag'] = table.authorized_flag.map({'Y':1,'N':0})
    
    table['category_1'] = table.category_1.map({'Y':1, 'N':0})
    
    most_appearing = table.category_2.value_counts().idxmax()
    table.category_2.fillna(value= most_appearing, inplace=True)
    table['enc_category_2_1'] = (table.category_2==1) + 0
    table['enc_category_2_2'] = (table.category_2==2) + 0
    table['enc_category_2_3'] = (table.category_2==3) + 0
    table['enc_category_2_4'] = (table.category_2==4) + 0
    table['enc_category_2_5'] = (table.category_2==5) + 0
    table.drop(columns = ['category_2'], inplace=True)

    most_appearing = table.category_3.value_counts().idxmax()
    table.category_3.fillna(value = most_appearing, inplace = True)
    table['enc_category_3_A'] = (table.category_3=='A') +0
    table['enc_category_3_B'] = (table.category_3=='B') +0
    table['enc_category_3_C'] = (table.category_3=='C') +0
    table.drop(columns = ['category_3'], inplace = True)
    
    table['enc_purchase_date'] = panda.DatetimeIndex(table.purchase_date).astype(np.int64)*1e-9
    table.drop(columns = ['purchase_date'], inplace = True)

    mean_installments = np.mean(table.installments.values)
    most_common_installment = table.installments.value_counts().idxmax()
    
    table['installments_mean_replaced'] = table['installments']
    table.installments_mean_replaced.replace(-1, mean_installments, inplace= True)
    table.installments_mean_replaced.replace(999, mean_installments, inplace= True)
    
    table['installments_common_replaced'] = table['installments']
    table.installments_mean_replaced.replace(-1, most_common_installment, inplace= True)
    table.installments_mean_replaced.replace(999, most_common_installment, inplace= True)
    
    table.drop(columns = ['installments'], inplace = True)
    
    
    
    

In [9]:
prep_for_aggregate(hist_transaction_data)

In [12]:
hist_transaction_data.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
authorized_flag,2.89739e+07,NaN,NaN,NaN,0.913768,0.280707,0,1,1,1,1
card_id,28973880,325540,C_ID_3d3dfdc692,5582,NaN,NaN,NaN,NaN,NaN,NaN,NaN
city_id,2.89739e+07,NaN,NaN,NaN,129.306,104.272,-1,53,90,212,347
category_1,2.89739e+07,NaN,NaN,NaN,0.0718127,0.258178,0,0,0,0,1
merchant_category_id,2.89739e+07,NaN,NaN,NaN,481.078,249.363,-1,307,454,705,891
merchant_id,28973880,326311,M_ID_00a6ca8a8a,1115097,NaN,NaN,NaN,NaN,NaN,NaN,NaN
month_lag,2.89739e+07,NaN,NaN,NaN,-4.50404,3.58772,-13,-7,-4,-2,0
purchase_amount,2.89739e+07,NaN,NaN,NaN,0.0342455,1126.08,-0.746908,-0.720311,-0.688304,-0.603029,6.0106e+06
state_id,2.89739e+07,NaN,NaN,NaN,10.5661,6.36784,-1,9,9,16,24
subsector_id,2.89739e+07,NaN,NaN,NaN,26.8491,9.68884,-1,19,29,34,41


In [21]:
## we will take the 
## sum,mean,max of purchase amount, 
## sum,mean,max of month lag by card_id
## mean of authorized_flag,
## num of authorized ==1
## num of authorized ==0
## sum, mean, max of installments
## mean of category_1
## sum of enc_category_2* and enc_category_3*

def count_authorized_one(x):
    return x.eq(1).sum()
def count_authorized_zero(x):
    return x.eq(0).sum()
def count_category_1_one(x):
    return x.eq(1).sum()
def count_category_1_zero(x):
    return x.eq(0).sum()

def count_of_unique_cities(x):
    return x.nunique()

def count_of_unique_subsector_id(x):
    return x.nunique()

def count_of_unique_state_id(x):
    return x.nunique()

def most_common_merchant_id(x):
    c = Counter(x)
    return c.most_common()[0][0]


In [22]:
aggregate = {
    
'purchase_amount' : ['sum','mean','max'],
'month_lag' : ['sum','mean','max'] ,
'enc_category_2_1':['sum'],
'enc_category_2_2': ['sum'],
'enc_category_2_3': ['sum'],
'enc_category_2_4': ['sum'],
'enc_category_2_5': ['sum'],    
'enc_category_3_A': ['sum'],
'enc_category_3_B': ['sum'],
'enc_category_3_C': ['sum'],
'installments_common_replaced': ['sum','mean','max'],
'installments_mean_replaced': ['sum','mean','max'],
'authorized_flag': ['mean', count_authorized_one, count_authorized_zero],
'category_1' :['mean', count_category_1_one, count_category_1_zero],
'enc_purchase_date' : ['mean','min','max',np.ptp],
'merchant_id' : [most_common_merchant_id]

    
}


In [23]:
group_by = hist_transaction_data.groupby(['card_id'])

In [24]:
len(hist_transaction_data)

28973880

In [25]:
t = group_by.agg(aggregate)

In [26]:
t.head()

purchase_amount                     month_lag                \
                            sum      mean       max       sum      mean max   
card_id                                                                       
C_ID_00007093c1      -76.845041 -0.515739  1.507069      -872 -5.852349   0   
C_ID_0001238066      -72.447201 -0.589002  0.768095      -223 -1.813008   0   
C_ID_0001506ef0      -34.601879 -0.524271  1.493545      -319 -4.833333   0   
C_ID_0001793786      -36.786013 -0.170306  4.554145      -719 -3.328704   0   
C_ID_000183fdda      -68.837938 -0.478041  2.764788      -353 -2.451389   0   

                enc_category_2_1 enc_category_2_2 enc_category_2_3  \
                             sum              sum              sum   
card_id                                                              
C_ID_00007093c1               28                0              120   
C_ID_0001238066              103                0                0   
C_ID_0001506ef0                2                0               64   
C_ID_0001793786              125               76               15   
C_ID_000183fdda               11                1              131   

                enc_category_2_4           ...                authorized_flag  \
                             sum           ...           count_authorized_one   
card_id                                    ...                                  
C_ID_00007093c1                0           ...                            114   
C_ID_0001238066                0           ...                            120   
C_ID_0001506ef0                0           ...                             62   
C_ID_0001793786                0           ...                            189   
C_ID_000183fdda                0           ...                            137   

                                      category_1                       \
                count_authorized_zero       mean count_category_1_one   
card_id                                                                 
C_ID_00007093c1                    35   0.187919                   28   
C_ID_0001238066                     3   0.016260                    2   
C_ID_0001506ef0                     4   0.000000                    0   
C_ID_0001793786                    27   0.009259                    2   
C_ID_000183fdda                     7   0.027778                    4   

                                      enc_purchase_date                \
                count_category_1_zero              mean           min   
card_id                                                                 
C_ID_00007093c1                   121      1.503054e+09  1.487081e+09   
C_ID_0001238066                   121      1.513977e+09  1.506638e+09   
C_ID_0001506ef0                    66      1.505646e+09  1.484411e+09   
C_ID_0001793786                   214      1.499412e+09  1.484994e+09   
C_ID_000183fdda                   140      1.512119e+09  1.502099e+09   

                                                      merchant_id  
                          max         ptp most_common_merchant_id  
card_id                                                            
C_ID_00007093c1  1.519708e+09  32627654.0         M_ID_9400cf2342  
C_ID_0001238066  1.519748e+09  13110825.0         M_ID_d17aabd756  
C_ID_0001506ef0  1.518871e+09  34460275.0         M_ID_b1fc88154d  
C_ID_0001793786  1.509481e+09  24487497.0         M_ID_923d57de8d  
C_ID_000183fdda  1.519592e+09  17492874.0         M_ID_f9cfe0a43b  

[5 rows x 31 columns]

In [27]:
t_cols=['card_id']
t_cols.extend(['_'.join(i) for i in t.columns.tolist()])

t_cols

['card_id',
 'purchase_amount_sum',
 'purchase_amount_mean',
 'purchase_amount_max',
 'month_lag_sum',
 'month_lag_mean',
 'month_lag_max',
 'enc_category_2_1_sum',
 'enc_category_2_2_sum',
 'enc_category_2_3_sum',
 'enc_category_2_4_sum',
 'enc_category_2_5_sum',
 'enc_category_3_A_sum',
 'enc_category_3_B_sum',
 'enc_category_3_C_sum',
 'installments_common_replaced_sum',
 'installments_common_replaced_mean',
 'installments_common_replaced_max',
 'installments_mean_replaced_sum',
 'installments_mean_replaced_mean',
 'installments_mean_replaced_max',
 'authorized_flag_mean',
 'authorized_flag_count_authorized_one',
 'authorized_flag_count_authorized_zero',
 'category_1_mean',
 'category_1_count_category_1_one',
 'category_1_count_category_1_zero',
 'enc_purchase_date_mean',
 'enc_purchase_date_min',
 'enc_purchase_date_max',
 'enc_purchase_date_ptp',
 'merchant_id_most_common_merchant_id']

In [28]:
t.reset_index(inplace=True)
t.columns = ['old_'+ i for i in t_cols]
t.head()

,old_card_id,old_purchase_amount_sum,old_purchase_amount_mean,old_purchase_amount_max,old_month_lag_sum,old_month_lag_mean,old_month_lag_max,old_enc_category_2_1_sum,old_enc_category_2_2_sum,old_enc_category_2_3_sum,...,old_authorized_flag_count_authorized_one,old_authorized_flag_count_authorized_zero,old_category_1_mean,old_category_1_count_category_1_one,old_category_1_count_category_1_zero,old_enc_purchase_date_mean,old_enc_purchase_date_min,old_enc_purchase_date_max,old_enc_purchase_date_ptp,old_merchant_id_most_common_merchant_id
0,C_ID_00007093c1,-76.845041,-0.515739,1.507069,-872,-5.852349,0,28,0,120,...,114,35,0.187919,28,121,1.503054e+09,1.487081e+09,1.519708e+09,32627654.0,M_ID_9400cf2342
1,C_ID_0001238066,-72.447201,-0.589002,0.768095,-223,-1.813008,0,103,0,0,...,120,3,0.016260,2,121,1.513977e+09,1.506638e+09,1.519748e+09,13110825.0,M_ID_d17aabd756
2,C_ID_0001506ef0,-34.601879,-0.524271,1.493545,-319,-4.833333,0,2,0,64,...,62,4,0.000000,0,66,1.505646e+09,1.484411e+09,1.518871e+09,34460275.0,M_ID_b1fc88154d
3,C_ID_0001793786,-36.786013,-0.170306,4.554145,-719,-3.328704,0,125,76,15,...,189,27,0.009259,2,214,1.499412e+09,1.484994e+09,1.509481e+09,24487497.0,M_ID_923d57de8d
4,C_ID_000183fdda,-68.837938,-0.478041,2.764788,-353,-2.451389,0,11,1,131,...,137,7,0.027778,4,140,1.512119e+09,1.502099e+09,1.519592e+09,17492874.0,M_ID_f9cfe0a43b


In [29]:
t.rename(columns = {'old_card_id':'card_id','old_merchant_id_most_common_merchant_id':'merchant_id'}, inplace=True)
t.head()

,card_id,old_purchase_amount_sum,old_purchase_amount_mean,old_purchase_amount_max,old_month_lag_sum,old_month_lag_mean,old_month_lag_max,old_enc_category_2_1_sum,old_enc_category_2_2_sum,old_enc_category_2_3_sum,...,old_authorized_flag_count_authorized_one,old_authorized_flag_count_authorized_zero,old_category_1_mean,old_category_1_count_category_1_one,old_category_1_count_category_1_zero,old_enc_purchase_date_mean,old_enc_purchase_date_min,old_enc_purchase_date_max,old_enc_purchase_date_ptp,merchant_id
0,C_ID_00007093c1,-76.845041,-0.515739,1.507069,-872,-5.852349,0,28,0,120,...,114,35,0.187919,28,121,1.503054e+09,1.487081e+09,1.519708e+09,32627654.0,M_ID_9400cf2342
1,C_ID_0001238066,-72.447201,-0.589002,0.768095,-223,-1.813008,0,103,0,0,...,120,3,0.016260,2,121,1.513977e+09,1.506638e+09,1.519748e+09,13110825.0,M_ID_d17aabd756
2,C_ID_0001506ef0,-34.601879,-0.524271,1.493545,-319,-4.833333,0,2,0,64,...,62,4,0.000000,0,66,1.505646e+09,1.484411e+09,1.518871e+09,34460275.0,M_ID_b1fc88154d
3,C_ID_0001793786,-36.786013,-0.170306,4.554145,-719,-3.328704,0,125,76,15,...,189,27,0.009259,2,214,1.499412e+09,1.484994e+09,1.509481e+09,24487497.0,M_ID_923d57de8d
4,C_ID_000183fdda,-68.837938,-0.478041,2.764788,-353,-2.451389,0,11,1,131,...,137,7,0.027778,4,140,1.512119e+09,1.502099e+09,1.519592e+09,17492874.0,M_ID_f9cfe0a43b


In [30]:
t.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
card_id,325540,325540,C_ID_60c1db665c,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
old_purchase_amount_sum,325540,NaN,NaN,NaN,3.04794,10624.7,-3595.35,-65.4392,-30.131,-12.3946,6.0106e+06
old_purchase_amount_mean,325540,NaN,NaN,NaN,1.49269,878.424,-0.74688,-0.67906,-0.621111,-0.500455,500883
old_purchase_amount_max,325540,NaN,NaN,NaN,54.7904,10622.9,-0.746878,-0.347954,0.124615,1.28017,6.0106e+06
old_month_lag_sum,325540,NaN,NaN,NaN,-400.871,614.593,-22477,-471,-178,-66,-1
old_month_lag_mean,325540,NaN,NaN,NaN,-3.85299,2.12658,-12.5,-5.53846,-3.5843,-2.04348,-0.0149254
old_month_lag_max,325540,NaN,NaN,NaN,-0.167214,0.636361,-11,0,0,0,0
old_enc_category_2_1_sum,325540,NaN,NaN,NaN,54.5107,87.1674,0,6,23,66,5579
old_enc_category_2_2_sum,325540,NaN,NaN,NaN,3.14361,22.9618,0,0,0,0,2551
old_enc_category_2_3_sum,325540,NaN,NaN,NaN,11.9543,42.3144,0,0,0,1,1374


In [31]:
del hist_transaction_data

In [32]:
t.to_csv('hist_with_merchant_aggregated.csv', index = False)

In [35]:
merchants_data_path = 'C:/TV Series/real_world_use_cases_1/supervised/regression/kaggle/elo/all_new/merchants_2.csv'
merchant_data = panda.read_csv(merchants_data_path)

In [37]:
merchant_data.shape, t.shape, t.card_id.unique().shape

((334696, 34), (325540, 32), (325540,))

In [46]:

# list(lambda x: 'merchant_'+x if not x.startswith('merchant') else x, merchant_data.columns.tolist())
merchant_data.columns = list(map(lambda x: 'merchant_'+x if not x.startswith('merchant') else x, merchant_data.columns.tolist()))

merchant_data.columns.tolist()

['merchant_id',
 'merchant_group_id',
 'merchant_category_id',
 'merchant_subsector_id',
 'merchant_numerical_1',
 'merchant_numerical_2',
 'merchant_category_1',
 'merchant_avg_sales_lag3',
 'merchant_avg_purchases_lag3',
 'merchant_active_months_lag3',
 'merchant_avg_sales_lag6',
 'merchant_avg_purchases_lag6',
 'merchant_active_months_lag6',
 'merchant_avg_sales_lag12',
 'merchant_avg_purchases_lag12',
 'merchant_active_months_lag12',
 'merchant_category_4',
 'merchant_city_id',
 'merchant_state_id',
 'merchant_category_2',
 'merchant_category_1_yes',
 'merchant_category_1_no',
 'merchant_category_4_yes',
 'merchant_category_4_no',
 'merchant_most_sales_range_E',
 'merchant_most_sales_range_D',
 'merchant_most_sales_range_C',
 'merchant_most_sales_range_B',
 'merchant_most_sales_range_A',
 'merchant_most_recent_purchases_range_E',
 'merchant_most_recent_purchases_range_D',
 'merchant_most_recent_purchases_range_C',
 'merchant_most_recent_purchases_range_B',
 'merchant_most_recent_pu

In [47]:
temp= panda.merge(t, merchant_data, on='merchant_id', how='left')
print(temp.shape)
temp.dropna(inplace=True)
print('after dropping emtpy:',temp.shape)

(347585, 65)
after dropping emtpy: (347585, 65)


In [48]:
temp.columns.tolist()

['card_id',
 'old_purchase_amount_sum',
 'old_purchase_amount_mean',
 'old_purchase_amount_max',
 'old_month_lag_sum',
 'old_month_lag_mean',
 'old_month_lag_max',
 'old_enc_category_2_1_sum',
 'old_enc_category_2_2_sum',
 'old_enc_category_2_3_sum',
 'old_enc_category_2_4_sum',
 'old_enc_category_2_5_sum',
 'old_enc_category_3_A_sum',
 'old_enc_category_3_B_sum',
 'old_enc_category_3_C_sum',
 'old_installments_common_replaced_sum',
 'old_installments_common_replaced_mean',
 'old_installments_common_replaced_max',
 'old_installments_mean_replaced_sum',
 'old_installments_mean_replaced_mean',
 'old_installments_mean_replaced_max',
 'old_authorized_flag_mean',
 'old_authorized_flag_count_authorized_one',
 'old_authorized_flag_count_authorized_zero',
 'old_category_1_mean',
 'old_category_1_count_category_1_one',
 'old_category_1_count_category_1_zero',
 'old_enc_purchase_date_mean',
 'old_enc_purchase_date_min',
 'old_enc_purchase_date_max',
 'old_enc_purchase_date_ptp',
 'merchant_id',


In [50]:
temp.columns = list(map(lambda x: x if (x.startswith('old') or x=='card_id') else 'old_'+x, temp.columns.tolist()))

In [51]:
temp.to_csv('hist_joined_with_merchants_data.csv',index = False)